# ETL Data with Python Project: Analyze User Behavior on TV and Digital Content Apps

## Raw data
This project involved developing an ETL pipeline using Python to analyze user behavior data from a TV viewing and digital content app. The raw data, stored in JSON format (approximately 4.5 million lines per file), was spread across four folders representing four days of user activity.
(However, I will analyze with the data lite version (first 1,000,000 rows of data) due to computer hardware limitations.)

<b> 1. Read data from source <b> <br>
<b> 2. Aggregate data <b> <br>
<b> 3. Merge for final OLAP output <b> <br>
<b> 4. EDA , Visualize <b> <br>
<b> 5. Import data to database / Export to excel <b> 

In [35]:
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt 
import os 
import datetime as dt

# Explore data folder

In [19]:
path = 'C:\\Users\\letha\\OneDrive - vn3nh\\Github\\DataProject\\ETL Data with Python - FPT user behavior\\Data\\'
os.listdir(path)

['20230126', '20230127', '20230128', '20230129']

There are 4 folder data for 4 days, from 26-01-2023 to 29-01-2023

# Understanding data

In [5]:
# df = pd.read_json(path+os.listdir(path)[3]+'//data.json',lines=True) # read raw data JSON file
file_path = path+'\\'+os.listdir(path)[3]+'\\data_lite.csv' # date lite version
file_path

'C:\\Users\\letha\\OneDrive - vn3nh\\Github\\DataProject\\ETL Data with Python - FPT user behavior\\Data\\20230129\\data_lite.csv'

In [7]:
df = pd.read_csv(file_path)


In [8]:
df.head()

,Unnamed: 0,receive_time,device_id,device,device_name,platform,isp,ItemName,ItemId,AppId,Event
0,0,2023-01-29 19:00:00,D4:CF:F9:A0:D3:81,BoxOTT,SEI500FPT,fplay-ottbox-sei-2020,fpt,Hà Nội - Hải Phòng,the-thao-1,IPTV,StartChannel
1,1,2023-01-29 19:00:00,d3f8cc063e08f35b,SmartTvAndroid,CASPER,smart-tv-normal,other,Hà Nội - Hải Phòng,the-thao-1,IPTV,StartChannel
2,2,2023-01-29 19:00:00,D4:CF:F9:A0:20:B0,BoxOTT,SEI500FPT,fplay-ottbox-sei-2020,fpt,Câu Chuyện Về Sấm Và Chớp,63526d33492c03ce16c2d840,home,StartMovie
3,3,2023-01-29 19:00:00,D97A5C6A-7506-4A9F-AE99-4AA5ABBE2D24,Mobile,iPhone,IOS,viettel,VTV1 HD,vtv1-hd,IPTV,StartChannel
4,4,2023-01-29 19:00:00,90412fb54a48561d,SmartTvAndroid,UnionTV,smarttv-tcl-android,viettel,VTV5 HD,vtv5-hd,IPTV,StartChannel


In [26]:
df.drop(df.columns[[0]], axis=1, inplace=True)

In [27]:
df.head()

,receive_time,device_id,device,device_name,platform,isp,ItemName,ItemId,AppId,Event
0,2023-01-29 19:00:00,D4:CF:F9:A0:D3:81,BoxOTT,SEI500FPT,fplay-ottbox-sei-2020,fpt,Hà Nội - Hải Phòng,the-thao-1,IPTV,StartChannel
1,2023-01-29 19:00:00,d3f8cc063e08f35b,SmartTvAndroid,CASPER,smart-tv-normal,other,Hà Nội - Hải Phòng,the-thao-1,IPTV,StartChannel
2,2023-01-29 19:00:00,D4:CF:F9:A0:20:B0,BoxOTT,SEI500FPT,fplay-ottbox-sei-2020,fpt,Câu Chuyện Về Sấm Và Chớp,63526d33492c03ce16c2d840,home,StartMovie
3,2023-01-29 19:00:00,D97A5C6A-7506-4A9F-AE99-4AA5ABBE2D24,Mobile,iPhone,IOS,viettel,VTV1 HD,vtv1-hd,IPTV,StartChannel
4,2023-01-29 19:00:00,90412fb54a48561d,SmartTvAndroid,UnionTV,smarttv-tcl-android,viettel,VTV5 HD,vtv5-hd,IPTV,StartChannel


In [28]:
df.dtypes

receive_time    object
device_id       object
device          object
device_name     object
platform        object
isp             object
ItemName        object
ItemId          object
AppId           object
Event           object
dtype: object

In [29]:
df.count()

receive_time    1000000
device_id        999951
device          1000000
device_name     1000000
platform         999999
isp             1000000
ItemName         999715
ItemId           999335
AppId            985852
Event           1000000
dtype: int64

In [34]:
print(df.isnull().sum())

receive_time        0
device_id          49
device              0
device_name         0
platform            1
isp                 0
ItemName          285
ItemId            665
AppId           14148
Event               0
dtype: int64


In [36]:
for i in df.columns:
    print("Checking column ",i)
    print(df[i].nunique())
    print(df[i].unique())

Checking column  receive_time
9482
['2023-01-29 19:00:00' '2023-01-29 19:00:01' '2023-01-29 19:00:02' ...
 '2023-01-29 18:00:56' '2023-01-29 18:00:57' '2023-01-29 18:00:58']
Checking column  device_id
225293
['D4:CF:F9:A0:D3:81' 'd3f8cc063e08f35b' 'D4:CF:F9:A0:20:B0' ...
 'D4:CF:F9:93:27:BC' 'D4:CF:F9:93:11:04' 'D4:CF:F9:92:E5:99']
Checking column  device
5
['BoxOTT' 'SmartTvAndroid' 'Mobile' 'SmartTV' 'WebOS']
Checking column  device_name
4566
['SEI500FPT' 'CASPER' 'iPhone' ... 'iPhone của ' 'Aland'
 'Pham-Viet’s iPhone']
Checking column  platform
22
['fplay-ottbox-sei-2020' 'smart-tv-normal' 'IOS' 'smarttv-tcl-android'
 'smarttv-ss-nextgen' 'smarttv-lg-nextgen' 'Android - Normal'
 'fplay-ottbox-hisense-2022' 'fplay-ottbox-sei-2021'
 'smarttv-sony-android' 'fplay-ottbox-sdmc-2020'
 'skyworth-smarttv-android' 'Windows' 'macOS' 'smart-tv-normal-no-drm'
 'vsmart' 'fplay-ottbox-sei-2022' 'fplay-gr-android-box'
 'smarttv-pana-android' 'Linux' 'Chrome OS' 'Tizen' nan]
Checking column  isp
5